In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import urllib

# Display Data about All Wildfire Incidents in Indonesia
Gather information for all wildfire events in Sumatra

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [3]:
wf_data  = pd.read_csv("../data-total.csv")
wf_data.columns
# rename columns
wf_data.rename(columns={'Lat': 'lat', 
                        'Long': 'long', 
                        'Satellite': 'satellite', 
                        'Time_UTC': 'time_utc', 
                        'Date': 'date', 
                        'Source': 'source', 
                        'PROVINSI': 'provinsi',
                        'KAB_KOTA': 'kab_kota'}, inplace=True)
# adding '-' to date column values
wf_data.date = pd.to_datetime(wf_data.date, format="%Y%m%d")
wf_data.date = wf_data.date.astype('string')

In [4]:
# All provinces in sumatera
sumatera_provinces = [
    'SUMATERA UTARA',
    'JAMBI',
    'RIAU',
    'SUMATERA BARAT',
    'SUMATERA SELATAN',
    'BENGKULU',
    'LAMPUNG',
    'ACEH'
]
# create filter so that only events in sumatera will be returned.
sumatera_filter = wf_data['provinsi'].apply(lambda x: x in sumatera_provinces) 
sumatera_only = wf_data.loc[sumatera_filter]

In [5]:
display(sumatera_only.info())
display(sumatera_only.provinsi.unique())
display(sumatera_only.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7839 entries, 1 to 39670
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   lat        7839 non-null   float64
 1   long       7839 non-null   float64
 2   satellite  7839 non-null   object 
 3   time_utc   7839 non-null   object 
 4   date       7839 non-null   string 
 5   source     7839 non-null   object 
 6   provinsi   7839 non-null   object 
 7   kab_kota   7839 non-null   object 
 8   kec2006    7830 non-null   object 
 9   desaa2006  7830 non-null   object 
 10  nama_kaw   930 non-null    object 
dtypes: float64(2), object(8), string(1)
memory usage: 734.9+ KB


None

array(['LAMPUNG', 'RIAU', 'SUMATERA UTARA', 'SUMATERA BARAT',
       'SUMATERA SELATAN', 'ACEH', 'JAMBI', 'BENGKULU'], dtype=object)

,lat,long,satellite,time_utc,date,source,provinsi,kab_kota,kec2006,desaa2006,nama_kaw
1,-5.06259,105.101,LPN-NPP,06:11:04 AM,2016-08-24,LAPAN,LAMPUNG,LAMPUNG TENGAH,ANAK TUHA,BUMI JAYA,NaN
4,1.18991,100.672,LPN-NPP,06:29:59 AM,2016-08-23,LAPAN,RIAU,ROKAN HULU,KECAMATAN KAPENUHAN,UPT SP IV KOTO TENGAH,NaN
5,2.31418,100.288,LPN-NPP,06:29:59 AM,2016-08-23,LAPAN,SUMATERA UTARA,LABUHANBATU,PANAI TENGAH,SELAT BETING,NaN
6,1.41833,100.752,LPN-NPP,06:29:59 AM,2016-08-23,LAPAN,RIAU,ROKAN HILIR,KECAMATAN PUJUD,SIARANG ARANG,NaN
8,1.38958,100.673,LPN-NPP,06:36:36 AM,2016-09-13,LAPAN,RIAU,ROKAN HILIR,KECAMATAN PUJUD,TELUK NAYANG,NaN


In [6]:
# changing the date data type to datetime
# Adding date day, month, and year to the data frame


if 'year' not in sumatera_only.columns.values:
    date_formatted = sumatera_only['date'].astype('datetime64[ns]')
    date = pd.DatetimeIndex(sumatera_only['date'])
    sumatera_only.insert(len(sumatera_only.columns), "year", date.year, True) # adding column
    sumatera_only.insert(len(sumatera_only.columns), "month", date.month, True) # adding column
    sumatera_only.insert(len(sumatera_only.columns), "day", date.day, True) # adding column


In [7]:
sumatera_only = sumatera_only.sort_values(by=['date'], 
                                          ascending=True,
                                          ignore_index=True)
sumatera_only


,lat,long,satellite,time_utc,date,source,provinsi,kab_kota,kec2006,desaa2006,nama_kaw,year,month,day
0,1.649680,100.892000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN BANGKO PUSAKO,SUNGAI MENASIB,NaN,2016,4,12
1,1.648530,100.884000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN BANGKO PUSAKO,SUNGAI MENASIB,NaN,2016,4,12
2,1.580030,100.855000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN TANAH PUTIH,BALAM SEMPURNA,NaN,2016,4,12
3,4.272360,97.747800,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,ACEH,ACEH TAMIANG,TAMIANG HULU,BATU BEDULANG,NaN,2016,4,12
4,1.656640,100.891000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN BANGKO PUSAKO,SUNGAI MENASIB,NaN,2016,4,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7834,-4.437215,104.505066,LPN-NPP,12:44:37 PM,2020-09-16,LAPAN,LAMPUNG,WAYKANAN,BLAMBANGAN UMPU,TANJUNG RAYA GIHAM,NaN,2020,9,16
7835,0.785336,99.191719,LPN-NPP,13:04:01 PM,2020-10-17,LAPAN,SUMATERA UTARA,MANDAILINGNATAL,NATAL,PARDAMEAN BARU,NaN,2020,10,17
7836,-0.470491,100.017670,LPN-NPP,13:04:01 PM,2020-10-17,LAPAN,SUMATERA BARAT,PADANGPARIAMAN,BATANG GASAN,MALAI BAWAH,NaN,2020,10,17
7837,1.306781,99.927521,LPN-NPP,13:04:01 PM,2020-10-17,LAPAN,SUMATERA UTARA,PADANG LAWAS,BARUMUN TENGAH,SIPAGABU,NaN,2020,10,17


# Import earth engine API which is called ee


In [8]:
import ee
import folium
from folium import plugins

In [9]:
# Authenticate earth engine servers
ee.Authenticate()

#initialize API
ee.Initialize()

Enter verification code: 4/1AdQt8qj8YdDq3XtxM4TzqMEIGtzuCWlVf1BCODvJVYOogWJhUbzxq7FNFlM

Successfully saved authorization token.


In [10]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.ImageCollection(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# cloud masking
# def cloudMask(collection_name):
#     remove_cloud = collection_name.mask(collection_name.select('QA_PIXEL').eq(2720))
#     return remove_cloud

def cloudMask(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20))\
                .addBands(image.metadata('system:time_start'))



# Satellite Imagery Discovery
Date must be long enough for the satellite to capture the image, e.g. 16 days, 24 days, or evenr 365.

# Landsat "LANDSAT/LC08/C02/T1_TOA"
USGS Landsat 8 Collection 2 Tier 1 TOA Reflectance


In [20]:
####################################################### lANDSAT #################################################################
# Select row column
row = 9

# Set coordinates
coordinates = [float(sumatera_only.lat.values[row]), float(sumatera_only.long.values[row])]
# coordinates = [37.3861, -122.0839]

# filter image collection
# set base date
base_date = ee.Date(sumatera_only.date.values[row])
region = ee.Geometry.Rectangle([coordinates[1]-0.2, coordinates[0]-0.2, 
                                coordinates[1]+0.2, coordinates[0]+0.2]);

# set geometry point
# point = ee.Geometry.Point(float(sumatera_only.long.values[row]), float(sumatera_only.lat.values[row]))
#################################################################################################################################


####################################################### PREFIRE #################################################################
# pre fire
prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

# landsat image pre wildfire event
pre_landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')\
                       .filterDate(prefire_date_start, prefire_date_end)\
                       .filterBounds(region)\
                       .map(cloudMask)\
                       .median()



# Calculate NBR for pre fire
pre_ndvi = pre_landsat_collection.normalizedDifference(['B5', 'B4']).gt(ee.Number(0.1)).rename('NDVI')
pre_landsat_collection = pre_landsat_collection.updateMask(pre_ndvi)
pre_nbr = pre_landsat_collection.normalizedDifference(['B5', 'B7'])
pre_ndwi = pre_landsat_collection.normalizedDifference(['B3', 'B5']).rename('NDWI')
pre_mndwi = pre_landsat_collection.normalizedDifference(['B2', 'B5']).rename('MNDWI')
check_pre = pre_landsat_collection.Not()


#################################################################################################################################


####################################################### POSTFIRE ################################################################
# post fire
postfire_date_start = ee.Date(base_date.advance(0, 'day'))
postfire_date_end = ee.Date(base_date.advance(365, 'day'))

# landsat image post wildfire event
post_landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')\
                       .filterDate(postfire_date_start, postfire_date_end)\
                       .filterBounds(region)\
                       .map(cloudMask)\
                       .median()
    
# post_landsat_collection = post_landsat_collection.qualityMosaic('system:time_start')

# Calculate NBR for post fire
post_ndvi = post_landsat_collection.normalizedDifference(['B5', 'B4']).gt(ee.Number(0.1)).rename('NDVI')
post_landsat_collection = post_landsat_collection.updateMask(post_ndvi)
post_nbr = post_landsat_collection.normalizedDifference(['B5', 'B7'])
post_ndwi = post_landsat_collection.normalizedDifference(['B2', 'B5']).rename('NDWI')
post_mndwi = post_landsat_collection.normalizedDifference(['B3', 'B5']).rename('MNDWI')
post_ndwi_mndwi = post_ndwi.add(post_mndwi).rename('I')


# calculate delta NBR
delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')

post_landsat_collection = post_landsat_collection\
                          .addBands(delta_nbr)\
                          .addBands(post_ndvi)\
                          .addBands(post_ndwi)\
                          .addBands(post_mndwi)\
                          .addBands(post_ndwi_mndwi)\
#################################################################################################################################


############################################## DISPLAY SATELLITE IMAGE ##########################################################
map_landsat = folium.Map(location=coordinates, zoom_start=11)
folium.Marker(coordinates, popup=sumatera_only.desaa2006).add_to(map_landsat)

# Delta NBR visualization
image_viz_params = {
    'bands': ['DELTA_NBR', 'B5', 'B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)

image_viz_params = {
    'bands': ['DELTA_NBR', 'B4', 'B3'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)

image_viz_params = {
    'bands': ['NDVI'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)

image_viz_params = {
    'bands': ['B4'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)

image_viz_params = {
    'bands': ['B3'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)

image_viz_params = {
    'bands': ['B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)

# Regular Image visualization 
image_viz_params = {
    'bands': ['B7', 'B5', 'B3'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(pre_landsat_collection, image_viz_params, None)
display(map_landsat)

# Regular Image visualization 
image_viz_params = {
    'bands': ['B7', 'B5', 'B3'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)
#################################################################################################################################

In [14]:
####################################################### lANDSAT #################################################################
# Select row column
row = 5003

# Set coordinates
coordinates = [float(sumatera_only.lat.values[row]), float(sumatera_only.long.values[row])]
# coordinates = [37.3861, -122.0839]

# filter image collection
# set base date
base_date = ee.Date(sumatera_only.date.values[row])
region = ee.Geometry.Rectangle([coordinates[1]-0.15, coordinates[0]-0.15, 
                                coordinates[1]+0.15, coordinates[0]+0.15]);

# set geometry point
# point = ee.Geometry.Point(float(sumatera_only.long.values[row]), float(sumatera_only.lat.values[row]))
#################################################################################################################################


####################################################### PREFIRE #################################################################
# pre fire
prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

# landsat image pre wildfire event
pre_landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')\
                       .filterDate(prefire_date_start, prefire_date_end)\
                       .filterBounds(region)\
                       .map(cloudMask)\
                       .median()

#                        .map(cloudMask2)\
#                        .median()



# Calculate NBR for pre fire
# pre_ndvi = pre_landsat_collection.normalizedDifference(['B5', 'B4']).gt(ee.Number(0.1)).rename('NDVI')
# pre_landsat_collection = pre_landsat_collection.updateMask(pre_ndvi)
pre_nbr = pre_landsat_collection.normalizedDifference(['B5', 'B7'])
# pre_ndwi = pre_landsat_collection.normalizedDifference(['B3', 'B5']).rename('NDWI')
# pre_mndwi = pre_landsat_collection.normalizedDifference(['B2', 'B5']).rename('MNDWI')


#################################################################################################################################


####################################################### POSTFIRE ################################################################
# post fire
postfire_date_start = ee.Date(base_date.advance(1, 'day'))
postfire_date_end = ee.Date(base_date.advance(365, 'day'))

# landsat image post wildfire event
post_landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')\
                       .filterDate(postfire_date_start, postfire_date_end)\
                       .filterBounds(region)\
                       .map(cloudMask)\
                       .median()

#                        .map(cloudMask2)\
#                        .median()
    
# post_landsat_collection = post_landsat_collection.qualityMosaic('system:time_start')

# Calculate NBR for post fire
# post_ndvi = post_landsat_collection.normalizedDifference(['B5', 'B4']).gt(ee.Number(0.1)).rename('NDVI')
# post_landsat_collection = post_landsat_collection.updateMask(post_ndvi)
post_nbr = post_landsat_collection.normalizedDifference(['B5', 'B7'])
# post_ndwi = post_landsat_collection.normalizedDifference(['B2', 'B5']).rename('NDWI')
# post_mndwi = post_landsat_collection.normalizedDifference(['B3', 'B5']).rename('MNDWI')
# post_ndwi_mndwi = post_ndwi.add(post_mndwi).rename('I')


# calculate delta NBR
delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
post_landsat_collection = post_landsat_collection\
                          .addBands(delta_nbr)\
#                           .addBands(post_ndvi)
#                           .addBands(post_ndwi)\
#                           .addBands(post_mndwi)\
#                           .addBands(post_ndwi_mndwi)\
#################################################################################################################################


############################################## DISPLAY SATELLITE IMAGE ##########################################################
map_landsat = folium.Map(location=coordinates, zoom_start=12)
folium.Marker(coordinates, popup=sumatera_only.desaa2006).add_to(map_landsat)

# Delta NBR visualization
image_viz_params = {
    'bands': ['DELTA_NBR', 'B4', 'B3'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)

image_viz_params = {
    'bands': ['B7', 'B5', 'B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)

image_viz_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)
# image_viz_params = {
#     'bands': ['NDVI'],
#     'min': 0,
#     'max': 0.4,
# }
# map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
# display(map_landsat)

# image_viz_params = {
#     'bands': ['B4'],
#     'min': 0,
#     'max': 0.4,
# }
# map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
# display(map_landsat)

# image_viz_params = {
#     'bands': ['B3'],
#     'min': 0,
#     'max': 0.4,
# }
# map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
# display(map_landsat)

# image_viz_params = {
#     'bands': ['B2'],
#     'min': 0,
#     'max': 0.4,
# }
# map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
# display(map_landsat)

# download_url = post_landsat_collection.getDownloadURL({
#     'region': region,
#     'bands' : ['DELTA_NBR', 'B4', 'B3'],
#     'scale' : 40,
#     'format': 'GeoTIFF'
# })

# # download_path = os.path.join(file_path)
# print(f"download data from {download_url}")
# # urllib.request.urlretrieve(download_url,'b8')
# print(f"{download_url} downloaded")

# # Regular Image visualization 
# image_viz_params = {
#     'bands': ['B7', 'B5', 'B3'],
#     'min': 0,
#     'max': 0.4,
# }
# map_landsat.add_ee_layer(pre_landsat_collection, image_viz_params, None)
# display(map_landsat)

# # Regular Image visualization 
# image_viz_params = {
#     'bands': ['B7', 'B5', 'B3'],
#     'min': 0,
#     'max': 0.4,
# }
# map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
# display(map_landsat)
#################################################################################################################################

EEException: Image.visualize: No band named 'NDVI'. Available band names: [B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11, QA_PIXEL, QA_RADSAT, SAA, SZA, VAA, VZA, system:time_start, DELTA_NBR].

# SENTINEL "COPERNICUS/S2"
Sentinel-2 MSI: MultiSpectral Instrument, Level-1C

# Collect all images and find its median composite image

In [18]:
#################################################################################################################################
# sentinel 2 cloud masking 
def s2CloudMasking(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)\
             .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask)\
                .divide(10000)\
#################################################################################################################################


####################################################### SENTINEL ################################################################
# Select row column
row = 5003


# Set coordinates
coordinates = [float(sumatera_only.lat.values[row]), float(sumatera_only.long.values[row])]
print(coordinates)
region = ee.Geometry.Rectangle([coordinates[1]-0.1, coordinates[0]-0.1, 
                                coordinates[1]+0.1, coordinates[0]+0.1]);


# filter image collection
# set base date
base_date = ee.Date(sumatera_only.date.values[row])

#################################################################################################################################


####################################################### PREFIRE #################################################################
# pre fire
prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

# sentinel image pre wildfire event
pre_sentinel_collection = ee.ImageCollection('COPERNICUS/S2')\
                           .filterDate(prefire_date_start, prefire_date_end)\
                           .filterBounds(region)\
                           .map(s2CloudMasking)\
                           .median()


# Calculate NBR for pre fire
pre_nbr = pre_sentinel_collection.normalizedDifference(['B8', 'B12'])
#################################################################################################################################


####################################################### POSTFIRE ################################################################
# post fire
postfire_date_start = ee.Date(base_date.advance(1, 'day'))
postfire_date_end = ee.Date(base_date.advance(365, 'day'))

# sentinel image post wildfire event
post_sentinel_collection = ee.ImageCollection('COPERNICUS/S2')\
                           .filterDate(postfire_date_start, postfire_date_end)\
                           .filterBounds(region)\
                           .map(s2CloudMasking)\
                           .median()
            

# Calculate NBR for post fire
post_nbr = post_sentinel_collection.normalizedDifference(['B8', 'B12'])
# post_ndwi = post_sentinel_collection.normalizedDifference(['B3', 'B8']).rename('NDWI')

# calculate delta NBR
delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
post_sentinel_collection = post_sentinel_collection.addBands(delta_nbr)
#################################################################################################################################


############################################## DISPLAY SATELLITE IMAGE ##########################################################
map_landsat = folium.Map(location=coordinates, zoom_start=12)
folium.Marker(coordinates, popup=sumatera_only.desaa2006).add_to(map_landsat)

# Delta NBR visualization
image_viz_params = {
    'bands': ['DELTA_NBR', 'B3', 'B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_sentinel_collection, image_viz_params, None)
display(map_landsat)

# Regular Image visualization 
image_viz_params = {
    'bands': ['B12', 'B8', 'B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(pre_sentinel_collection, image_viz_params, None)
display(map_landsat)

# Regular Image visualization 
image_viz_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(pre_sentinel_collection, image_viz_params, None)
display(map_landsat)

# # Regular Image visualization 
# image_viz_params = {
#     'bands': ['NDWI', 'B8', 'B4'],
#     'min': 0,
#     'max': 0.4,
# }
map_landsat.add_ee_layer(post_sentinel_collection, image_viz_params, None)
display(map_landsat)
#################################################################################################################################
region = ee.Geometry.Rectangle([coordinates[1]-0.1, coordinates[0]-0.1, 
                                coordinates[1]+0.1, coordinates[0]+0.1]);

# file_path = 'D:\\wildfire-sumatera-dataset\\wildfire-sumatera-geotiff\\sentinel-2\\postfire\\2283'

# os.chdir(file_path)

# download_url = post_sentinel_collection.getDownloadURL({
#     'region': region,
#     'bands' : ['DELTA_NBR'],
#     'scale' : 50,
#     'format': 'GeoTIFF'
# })

# download_path = os.path.join(file_path)
# print(f"download data from {download_url}")
# urllib.request.urlretrieve(download_url,'b8.tif')
# print(f"{download_url} downloaded")
# # import urllib
# file_path = "C:\\Users\\USER\\Downloads\\aa"
# urllib.request.urlretrieve(download_url, file_path)

[-3.37412, 105.232]


In [ ]:
#################################################################################################################################
# sentinel 2 cloud masking 
def s2CloudMasking(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)\
             .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask)\
                .divide(10000)\
#################################################################################################################################


####################################################### SENTINEL ################################################################
# Select row column
row = 9


# Set coordinates
coordinates = [float(sumatera_only.lat.values[row]), float(sumatera_only.long.values[row])]
region = ee.Geometry.Rectangle([coordinates[1]-0.1, coordinates[0]-0.1, 
                                coordinates[1]+0.1, coordinates[0]+0.1]);


# filter image collection
# set base date
base_date = ee.Date(sumatera_only.date.values[row])

#################################################################################################################################


####################################################### PREFIRE #################################################################
# pre fire
prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

# sentinel image pre wildfire event
pre_sentinel_collection = ee.ImageCollection('COPERNICUS/S2')\
                           .filterDate(prefire_date_start, prefire_date_end)\
                           .filterBounds(region)\
                           .map(s2CloudMasking)\
                           .median()


# Calculate NBR for pre fire
pre_nbr = pre_sentinel_collection.normalizedDifference(['B8', 'B12'])
#################################################################################################################################


####################################################### POSTFIRE ################################################################
# post fire
postfire_date_start = ee.Date(base_date.advance(1, 'day'))
postfire_date_end = ee.Date(base_date.advance(365, 'day'))

# sentinel image post wildfire event
post_sentinel_collection = ee.ImageCollection('COPERNICUS/S2')\
                           .filterDate(postfire_date_start, postfire_date_end)\
                           .filterBounds(region)\
                           .map(s2CloudMasking)\
                           .median()
            

# Calculate NBR for post fire
post_nbr = post_sentinel_collection.normalizedDifference(['B8', 'B12'])
post_ndwi = post_sentinel_collection.normalizedDifference(['B3', 'B8']).rename('NDWI')

# calculate delta NBR
delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
post_sentinel_collection = post_sentinel_collection.addBands(delta_nbr).addBands(post_ndwi)
#################################################################################################################################


############################################## DISPLAY SATELLITE IMAGE ##########################################################
map_landsat = folium.Map(location=coordinates, zoom_start=12)
folium.Marker(coordinates, popup=sumatera_only.desaa2006).add_to(map_landsat)

# Delta NBR visualization
image_viz_params = {
    'bands': ['DELTA_NBR', 'B3', 'B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_sentinel_collection, image_viz_params, None)
display(map_landsat)

# Regular Image visualization 
image_viz_params = {
    'bands': ['B12', 'B8', 'B4'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(pre_sentinel_collection, image_viz_params, None)
display(map_landsat)

# Regular Image visualization 
image_viz_params = {
    'bands': ['NDWI', 'B8', 'B4'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_sentinel_collection, image_viz_params, None)
display(map_landsat)
#################################################################################################################################
region = ee.Geometry.Rectangle([coordinates[1]-0.1, coordinates[0]-0.1, 
                                coordinates[1]+0.1, coordinates[0]+0.1]);

# file_path = 'D:\\wildfire-sumatera-dataset\\wildfire-sumatera-geotiff\\sentinel-2\\postfire\\2283'

# os.chdir(file_path)

# download_url = post_sentinel_collection.getDownloadURL({
#     'region': region,
#     'bands' : ['DELTA_NBR'],
#     'scale' : 50,
#     'format': 'GeoTIFF'
# })

# download_path = os.path.join(file_path)
# print(f"download data from {download_url}")
# urllib.request.urlretrieve(download_url,'b8.tif')
# print(f"{download_url} downloaded")
# # import urllib
# file_path = "C:\\Users\\USER\\Downloads\\aa"
# urllib.request.urlretrieve(download_url, file_path)

In [28]:
os.getcwd()

'C:\\Users\\USER\\Documents\\DATA USER\\DATA D\\skripsi-uph\\wildfire-data-science\\google-earth-engine'

# Download the photo

In [ ]:
def getUrl(satellite_image, region, bands, scale, file_format):
    '''
    This function is aimed for returning URL for downloading the sattelite image which receives arguments
    
    - sattelite_image which receives an argument type of ee.Image
    - region which receives an argument type of ee.Region
    - band which receives bands of of the sattelite in form of an array of strings
    - scale which receives scale in meters (ee.Number)
    - format which receives file format (String)
    
    '''
    image_url = satellite_image.getDownloadURL({
        'region': region,
        'bands' : bands,
        'scale' : scale,
        'format': file_format})
    return image_url
        


region = ee.Geometry.Rectangle([coordinates[1]-0.2, coordinates[0]-0.2, 
                                coordinates[1]+0.2, coordinates[0]+0.2]);

download_url = getUrl(satellite_image=pre_sentinel_collection, 
                      region=region,
                      bands = ["B4", "B3", "B2"],
                      scale=30,
                      file_format="GEO_TIFF"
                     )


# Download Sattelite Image 

In [ ]:


# # def download_satellite_images(satellite_name, dataset, bands, scale, file_format):
# #     for row in range(2):
# #         # Set coordinates
# #         coordinates = [float(dataset.lat.values[row]), float(dataset.long.values[row])]
# #         region = ee.Geometry.Rectangle([coordinates[1]-0.3, coordinates[0]-0.3, 
# #                                         coordinates[1]+0.3, coordinates[0]+0.3]);


# #         # filter image collection
# #         # set base date
# #         base_date = ee.Date(dataset.date.values[row])

# #         # set geometry point
# #         point = ee.Geometry.Point(dataset.long.values[row], dataset.lat.values[row])

# #         # pre fire
# #         prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
# #         prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

# #         # sentinel image pre wildfire event
# #         pre_sentinel_collection = ee.ImageCollection(satellite_name)\
# #                                    .filterDate(prefire_date_start, prefire_date_end)\
# #                                    .filterBounds(region)\
# #                                    .map(s2CloudMasking)\
# #                                    .median()

# #         # Calculate NBR for pre fire
# #         pre_nbr = pre_sentinel_collection.normalizedDifference(['B8', 'B12'])


# #         ####################################################### POSTFIRE ################################################################
# #         # post fire
# #         postfire_date_start = ee.Date(base_date.advance(1, 'day'))
# #         postfire_date_end = ee.Date(base_date.advance(365, 'day'))

# #         # sentinel image post wildfire event
# #         post_sentinel_collection = ee.ImageCollection(satellite_name)\
# #                                    .filterDate(postfire_date_start, postfire_date_end)\
# #                                    .filterBounds(region)\
# #                                    .map(s2CloudMasking)\
# #                                    .median()


# #         # Calculate NBR for post fire
# #         post_nbr = post_sentinel_collection.normalizedDifference(['B8', 'B12'])

# #         # calculate delta NBR
# #         delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
# #         post_sentinel_collection = post_sentinel_collection.addBands(delta_nbr)

# #         # select region of the satellite image to be downloaded
# #         region = ee.Geometry.Rectangle([coordinates[1]-0.2, coordinates[0]-0.2, 
# #                                     coordinates[1]+0.2, coordinates[0]+0.2]);

# #         # prefire URL
# #         prefire_download_url = pre_sentinel_collection.getDownloadURL({
# #             'region': region,
# #             'bands' : bands,
# #             'scale' : scale,
# #             'format': file_format
# #         })

# #         # postfire URL
# #         postfire_download_url = post_sentinel_collection.getDownloadURL({
# #             'region': region,
# #             'bands' : bands,
# #             'scale' : scale,
# #             'format': file_format
# #         })
# #         print(f'pre {row}: {prefire_download_url}')
# #         print(f'post {row}: {postfire_download_url}')
        

# def get_prefire_image_sentinel_satellite_urls(satellite_name, dataset, bands, scale, file_format):
    
#     url_list = [[]]
    
#     for j in range(2):
#         # Set coordinates
#         coordinates = [float(dataset.lat.values[j]), float(dataset.long.values[j])]
#         region = ee.Geometry.Rectangle([coordinates[1]-0.2, coordinates[0]-0.2, 
#                                         coordinates[1]+0.2, coordinates[0]+0.2]);

#         # filter image collection
#         # set base date
#         base_date = ee.Date(dataset.date.values[j])

#         # pre fire
#         prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
#         prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

#         # sentinel image pre wildfire event
#         prefire_collection = ee.ImageCollection(satellite_name)\
#                                    .filterDate(prefire_date_start, prefire_date_end)\
#                                    .filterBounds(region)\
#                                    .map(s2CloudMasking)\
#                                    .median()

#         # Calculate NBR for pre fire
#         pre_nbr = prefire_collection.normalizedDifference(['B8', 'B12'])

#         # select region of the satellite image to be downloaded
#         region = ee.Geometry.Rectangle([coordinates[1]-0.2, coordinates[0]-0.2, 
#                                     coordinates[1]+0.2, coordinates[0]+0.2])
        
#         # prefire URL
#         prefire_download_url = prefire_collection.getDownloadURL({
#             'region': region,
#             'bands' : bands,
#             'scale' : scale,
#             'format': file_format
#         })
        
#         url_list.append(prefire_download_url)

        
#     return url_list

# Download Sentinel Sattelite Image

In [ ]:
def get_sentinel_satellite_image_urls(satellite_name, situation, dataset, scale, file_format):
    url_list = []
    
    for row in range(5):
        # Set coordinates
        coordinates = [float(dataset.lat.values[row]), float(dataset.long.values[row])]
        region = ee.Geometry.Rectangle([coordinates[1]-0.2, coordinates[0]-0.2, 
                                        coordinates[1]+0.2, coordinates[0]+0.2]);


        # filter image collection
        # set base date
        base_date = ee.Date(dataset.date.values[row])

        # set geometry point
        point = ee.Geometry.Point(dataset.long.values[row], dataset.lat.values[row])

        # pre fire
        prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
        prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

        # sentinel image pre wildfire event
        prefire_collection = ee.ImageCollection(satellite_name)\
                                   .filterDate(prefire_date_start, prefire_date_end)\
                                   .filterBounds(region)\
                                   .map(s2CloudMasking)\
                                   .median()

        # Calculate NBR for pre fire
        pre_nbr = prefire_collection.normalizedDifference(['B8', 'B12'])


        # post fire
        postfire_date_start = ee.Date(base_date.advance(1, 'day'))
        postfire_date_end = ee.Date(base_date.advance(365, 'day'))

        # sentinel image post wildfire event
        postfire_collection = ee.ImageCollection(satellite_name)\
                                   .filterDate(postfire_date_start, postfire_date_end)\
                                   .filterBounds(region)\
                                   .map(s2CloudMasking)\
                                   .median()

        # Calculate NBR for post fire
        post_nbr = postfire_collection.normalizedDifference(['B8', 'B12'])

        # calculate delta NBR
        delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
        postfire_collection = postfire_collection.addBands(delta_nbr)
        
        if situation == 'prefire':
            download_url = prefire_collection.getDownloadURL({
                'region': region,
                'bands' : ['B12', 'B8', 'B4', 'B3', 'B2'],
                'scale' : scale,
                'format': file_format
            })

        else:
            download_url = postfire_collection.getDownloadURL({
                'region': region,
                'bands' : ['DELTA_NBR', 'B12', 'B8', 'B4', 'B3', 'B2'],
                'scale' : scale,
                'format': file_format
            })
        
        url_list.append(download_url)
        
        
    return url_list

# Download Landsat Sattelite Image 

In [3]:
def get_landsat_satellite_image_urls(satellite_name, situation, dataset, scale, file_format):
    url_list = []
    
    for row in range(5):
        # Set coordinates
        coordinates = [float(dataset.lat.values[row]), float(dataset.long.values[row])]
        region = ee.Geometry.Rectangle([coordinates[1]-0.2, coordinates[0]-0.2, 
                                        coordinates[1]+0.2, coordinates[0]+0.2]);


        # filter image collection
        # set base date
        base_date = ee.Date(dataset.date.values[row])

        # set geometry point
        point = ee.Geometry.Point(dataset.long.values[row], dataset.lat.values[row])

        # pre fire
        prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
        prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

        # landsat image pre wildfire event
        prefire_collection = ee.ImageCollection(satellite_name)\
                                   .filterDate(prefire_date_start, prefire_date_end)\
                                   .filterBounds(region)\
                                   .map(cloudMask2)\
                                   .median()

        # Calculate NBR for pre fire
        pre_nbr = prefire_collection.normalizedDifference(['B5', 'B7'])


        # post fire
        postfire_date_start = ee.Date(base_date.advance(1, 'day'))
        postfire_date_end = ee.Date(base_date.advance(365, 'day'))

        # landsat image post wildfire event
        postfire_collection = ee.ImageCollection(satellite_name)\
                                   .filterDate(postfire_date_start, postfire_date_end)\
                                   .filterBounds(region)\
                                   .map(cloudMask2)\
                                   .median()

        # Calculate NBR for post fire
        post_nbr = postfire_collection.normalizedDifference(['B5', 'B7'])

        # calculate delta NBR
        delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
        postfire_collection = postfire_collection.addBands(delta_nbr)

        if situation == 'prefire':
            download_url = prefire_collection.getDownloadURL({
                'region': region,
                'bands' : ['B7', 'B5', 'B4', 'B3', 'B2'],
                'scale' : scale,
                'format': file_format
            })
        
        else:
            download_url = postfire_collection.getDownloadURL({
                'region': region,
                'bands' : ['DELTA_NBR', 'B7', 'B5', 'B4', 'B3', 'B2'],
                'scale' : scale,
                'format': file_format
            })
            
        url_list.append(download_url)
    
    return url_list

In [ ]:
# create folder fot .tiff files
current_file_path= r"C:\Users\USER\Documents\DATA USER\DATA D\skripsi-uph\wildfire-data-science\google-earth-engine"

if os.getcwd() is not current_file_path:
    print("Directory has been reset to this file path")

parent_dir = os.path.abspath(os.path.join(current_file_path, os.pardir))
os.chdir(parent_dir)
print(os.getcwd())

In [ ]:
# create folder for sentinel-2 .tiff files
sentinel_prefire_tiff_folder = parent_dir + r"\wildfire-indonesia-sentinel-2\prefire"
sentinel_postfire_tiff_folder = parent_dir + r"\wildfire-indonesia-sentinel-2\postfire"

# create folder for landsat-8 .tiff files
landsat_prefire_tiff_folder = parent_dir + r"\wildfire-indonesia-landsat-8\prefire"
landsat_postfire_tiff_folder = parent_dir + r"\wildfire-indonesia-landsat-8\postfire"


# check whether the folder is exists
if not os.path.exists(sentinel_prefire_tiff_folder):
    os.makedirs(sentinel_prefire_tiff_folder)
    print(f"{sentinel_prefire_tiff_folder} has been successfully create")

if not os.path.exists(sentinel_postfire_tiff_folder):
    os.makedirs(sentinel_postfire_tiff_folder)
    print(f"{sentinel_postfire_tiff_folder} has been successfully create")
    
# check whether the folder is exists
if not os.path.exists(landsat_prefire_tiff_folder):
    os.makedirs(landsat_prefire_tiff_folder)
    print(f"{landsat_prefire_tiff_folder} has been successfully create")

if not os.path.exists(landsat_postfire_tiff_folder):
    os.makedirs(landsat_postfire_tiff_folder)
    print(f"{landsat_postfire_tiff_folder} has been successfully create")

In [ ]:
# get list of url path for downloading .tiff files
sentinel_prefire_url_list = get_sentinel_satellite_image_urls(satellite_name='COPERNICUS/S2',
                                                             situation='prefire',                                                                     
                                                             dataset=sumatera_only,
                                                             scale=35,
                                                             file_format='GeoTIFF'
                                                            )
print("done")

sentinel_postfire_url_list = get_sentinel_satellite_image_urls(satellite_name='COPERNICUS/S2',
                                                               situation='postfire',
                                                               dataset=sumatera_only,
                                                               scale=35,
                                                               file_format='GeoTIFF'
                                                               )
print("done")

landsat_prefire_url_list = get_landsat_satellite_image_urls(satellite_name='LANDSAT/LC08/C02/T1_TOA',
                                                             situation='prefire',                                                                     
                                                             dataset=sumatera_only,
                                                             scale=35,
                                                             file_format='GeoTIFF'
                                                            )
print("done")

landsat_postfire_url_list = get_landsat_satellite_image_urls(satellite_name='LANDSAT/LC08/C02/T1_TOA',
                                                             situation='postfire',                                                                     
                                                             dataset=sumatera_only,
                                                             scale=35,
                                                             file_format='GeoTIFF'
                                                            )
print("done")

print(sentinel_prefire_url_list)
print(sentinel_postfire_url_list)
print(landsat_prefire_url_list)
print(landsat_postfire_url_list)

In [ ]:
def get_filename(url):
    """
    Parses filename from given url
    """
    if url.find('/'):
        temp = url.rsplit('/', 1)[1]
        return temp.rsplit(':', 1)[0]
    
    
def download_from_url_link(url_list, folder_path):
    """
    Download file using based on array of urls from parameter url_list
    """
    for i in range(len(url_list)):
#         file_name = get_filename(url)
        file_name = str(i)
        file_path = os.path.join(folder_path, file_name)

        if not os.path.exists(file_path):
            print("Downloading...", file_name)
            urllib.request.urlretrieve(url_list[i], file_path)

    print('done')



In [ ]:
# download_from_url_link(sentinel_prefire_url_list, sentinel_prefire_tiff_folder)
download_from_url_link(sentinel_postfire_url_list, sentinel_postfire_tiff_folder)
# download_from_url_link(landsat_prefire_url_list, landsat_prefire_tiff_folder)
# download_from_url_link(landsat_postfire_url_list, landsat_postfire_tiff_folder)

In [ ]:
os.getcwd()

In [ ]:
sentinel_postfire_url_list